# SQLAlchemy Challenge

### Dependencies and Setup

In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import pandas as pd
import datetime as dt
pd.set_option('display.max_colwidth', 40)
pd.options.display.float_format = '{:,.2f}'.format

## Database Connection

In [35]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import relationship
from sqlalchemy import *

In [28]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

## Datasets Overview

In [4]:
measurement = Base.classes.measurement
session = Session(engine)
for measure in session.query(measurement).limit(10):
    print(measure.id, measure.station, measure.date, measure.prcp, measure.tobs)

1 USC00519397 2010-01-01 0.08 65.0
2 USC00519397 2010-01-02 0.0 63.0
3 USC00519397 2010-01-03 0.0 74.0
4 USC00519397 2010-01-04 0.0 76.0
5 USC00519397 2010-01-06 None 73.0
6 USC00519397 2010-01-07 0.06 70.0
7 USC00519397 2010-01-08 0.0 64.0
8 USC00519397 2010-01-09 0.0 68.0
9 USC00519397 2010-01-10 0.0 73.0
10 USC00519397 2010-01-11 0.01 64.0


In [5]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
table_1 = inspector.get_table_names()[0]
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns(table_1)
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [6]:
measurement = pd.read_sql('measurement', engine)
print(measurement.shape)
measurement.head()

(19550, 5)


,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.00
1,2,USC00519397,2010-01-02,0.00,63.00
2,3,USC00519397,2010-01-03,0.00,74.00
3,4,USC00519397,2010-01-04,0.00,76.00
4,5,USC00519397,2010-01-06,nan,73.00


In [7]:
station = pd.read_sql('station', engine)
print(station.shape)
station.head()

(9, 6)


,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27,-157.82,3.00
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42,-157.80,14.60
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52,-157.84,7.00
3,4,USC00517948,"PEARL CITY, HI US",21.39,-157.98,11.90
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.50,-158.01,306.60


## Reflect Tables into SQLAlchemy ORM

In [37]:
# reflect an existing database into a new model

# reflect the tables
class Measurement(Base):
        __tablename__ = 'measurement'
 
        # override schema elements like Columns
        date = Column('date', Date)
        station = relationship("station", collection_class=set)

engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
station = Base.classes.station


InvalidRequestError: Table 'measurement' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [38]:
u1 = session.query(Measurement).first()
u1.station

UnmappedClassError: Class __main__.Measurement is a subclass of AutomapBase.  Mappings are not produced until the .prepare() method is called on the class hierarchy.

In [32]:
# We can view all of the classes that automap found
Base.metadata.create_all(engine)

In [10]:
# Save references to each table


In [11]:
# Create our session (link) from Python to the DB


# Exploratory Climate Analysis

In [12]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [13]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [14]:
# Design a query to show how many stations are available in this dataset?


In [15]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [16]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [17]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [18]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

[(62.0, 69.57142857142857, 74.0)]


In [19]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [20]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [21]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [22]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

In [23]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [24]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [25]:
# Plot the daily normals as an area plot with `stacked=False`
